In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import random
import scipy.stats as spstats
import tqdm

import netdelib
import netdelib.topologies.topologies as ndtopo
import repsci

%matplotlib inline

plt.rcParams['font.size'] = 18
plt.rcParams['axes.linewidth'] = 2

In [ ]:
def broadcast_time_residue(N=100, M=5, runs=1):
    total = 0
    all_times = []
    for run in range(runs):
        times = dict()
        visited = [{v} for v in range(N)]
        stage = 0
        while sum([len(l) for l in visited]) < N * N:
            # Generate groups
            pods = ndtopo.get_long_path_stage_groups(N, M, stage, lowest=M)
            stage += 1
            # Create neighbor tables
            neighbors = dict((v, set()) for v in range(N))
            for pod in pods:
                for v in pod:
                    neighbors[v] = set(pod) - set([v])
            # Propagate broadcast for each node
            for v in range(N):
                # Skip if broadcast is complete
                if v in times:
                    continue
                # Iterate through previously reached nodes
                received = set()
                for w in visited[v]:
                    # Broadcast to their neighbors
                    received |= neighbors[w]
                visited[v] |= received
                if len(visited[v]) == N:
                    times[v] = stage
        all_times += times.values()
    return np.mean(all_times), np.std(all_times, ddof=1)

In [ ]:
def broadcast_time_random(N=100, M=5, runs=1):
    total = 0
    all_times = []
    for run in range(runs):
        times = dict()
        visited = [{v} for v in range(N)]
        stage = 0
        while sum([len(l) for l in visited]) < N * N:
            # Generate groups
            pods = ndtopo.get_random_stage_groups(N, M, stage)
            stage += 1
            # Create neighbor tables
            neighbors = dict((v, set()) for v in range(N))
            for pod in pods:
                for v in pod:
                    neighbors[v] = set(pod) - set([v])
            # Propagate broadcast for each node
            for v in range(N):
                # Skip if broadcast is complete
                if v in times:
                    continue
                # Iterate through previously reached nodes
                received = set()
                for w in visited[v]:
                    # Broadcast to their neighbors
                    received |= neighbors[w]
                visited[v] |= received
                if len(visited[v]) == N:
                    times[v] = stage
        all_times += times.values()
    return np.mean(all_times), np.std(all_times, ddof=1)

In [ ]:
experiment = repsci.Experiment('broadcast')

In [ ]:
residue_results = []
random_results = []
for n in tqdm.tqdm(range(50, 5000, 10)):
    residue_results.append(broadcast_time_residue(N=n,M=5,runs=1))
    random_results.append(broadcast_time_random(N=n,M=5,runs=100))

In [ ]:
out = experiment.get_filename('longpath.npy')
np.save(out, residue_results)
out = experiment.get_filename('random.npy')
np.save(out, random_results)

In [ ]:
plt.figure(figsize=(6.66,4.44))

y_res, s = zip(*residue_results)
x = range(50, 5000, 10)[:len(x_res)]
plt.plot(x, y_res, 'b', label='Long-Path')

y_ran, s = zip(*random_results)
err95 = 2 * np.divide(s, 10)
plt.fill_between(x, y_ran-err95, y_ran+err95, color="#cccccc")
plt.plot(x, y_ran, 'g', label='Random Pod')

plt.xlim([0, 3500])
plt.ylim([0, 30])

plt.grid()
plt.legend()

plt.xlabel('Number of Agents')
plt.ylabel('Mean Broadcast Time')
plt.tight_layout()

out = experiment.get_filename('broadcast.png')
plt.savefig(out, dpi=600)
out = experiment.get_filename('broadcast.eps')
plt.savefig(out, dpi=600)

In [ ]:
plt.figure()

y_res, s = zip(*residue_results)
x = range(50, 5000, 10)[:len(x_res)]

y_ran, s = zip(*random_results)
err95 = 2 * np.divide(s, 10)
plt.plot(y_res, y_ran, 'g')
plt.fill_between(y_res, y_ran-err95, y_ran+err95, color="#dddddd")

plt.xscale('log')
plt.grid()

plt.xlabel('Mean Broadcast Time (Long-Path)')
plt.ylabel('Mean Broadcast Time (Random Pod)' )

plt.tight_layout()
out = experiment.get_filename('broadcast-yy.png')
plt.savefig(out, dpi=600)
out = experiment.get_filename('broadcast-yy.eps')
plt.savefig(out, dpi=600)